# TRABALHO 2 DE REDES

#### Nomes:
* Augusto Ferreira
* Bruno Moreira
* Gabriel Martorano
* Rodolpho
* Débora ✝

#### Professor: 
* Luiz Henrique

### Classe Socket

    * initsocket: inicia o socket
    * send: envia a mensagem
    * recive: recebe a mensagem
   

In [1]:
import socket


class MySocket:

    def __init__(self, host=socket.getfqdn(socket.gethostbyname(socket.gethostname())), port=6069):
        self.__host = host
        self.__port = port

    @staticmethod
    def initsocket():
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        return s

    def send(self, msg):
        s = self.initsocket()

        s.connect((self.__host, self.__port))
        s.sendall(msg.encode())
        # data = s.recv(1024)
        s.close()

    def receive(self):
        s = self.initsocket()
        s.bind((self.__host, self.__port))
        s.listen(1)
        data = ''
        received = []
        # recebe mensagem
        while data != b'#':
            # recebe conexao
            conn, addr = s.accept()
            data = conn.recv(3000)
            if not data:
                continue
            conn.close()
            received.append(repr(data))

        received.pop()
        return (received)


### Classe DataFrame

    * set__crc: define o crc
    * last_bit_sequence: retorna ultimo bit da sequencia
    * first_bit_sequence: retorna o primeiro bit da sequencia
    * return_data_frame: retorna o data frame em uma string
    * return_data_drame_to_crc: retorna o data frame um uma string 
    * string_to_DataFrame: retorna o objeto dataFrame

In [2]:
class DataFrame:
    __delimiter = None
    __length = None
    __sequence = None
    __destination_address = None
    __source_address = None
    __payload = None
    __crc = None

    def __init__(self, length, sequence, destination_address, source_address, payload):
        self.__delimiter = "01111110"
        self.__length = length
        self.__sequence = sequence
        self.__destination_address = destination_address
        self.__source_address = source_address
        self.__payload = payload
        self.__crc = ""

    def set__crc(self, crc):
        self.__crc = crc

    def last_bit_sequence(self):
        return self.__sequence[7]

    def first_bit_sequence(self):
        return self.__sequence[0]

    def return_data_frame(self):
        frame = ''
        frame += self.__delimiter
        frame += self.__length
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address
        frame += self.__payload
        frame += self.__crc
        return frame

    def return_data_frame_to_crc(self):
        frame = ''
        frame += self.__length
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address
        frame += self.__payload
        return frame

    @staticmethod
    def string_to_DataFrame(string):
        aux = len(string)
        delimiter = string[0:8]
        length = string[9:16]
        sequence = string[17:24]
        destination_address = string[25:56]
        source_address = string[57:88]
        payload = string[89:aux - 16 - 1]
        crc = string[aux - 16:aux]
        dataFrame = DataFrame(length, sequence, destination_address, source_address, payload)
        dataFrame.set__crc(crc)

        return dataFrame

    def get_payload(self):
        return self.__payload


### Classe ConfirmationFrame

    * last_bit_sequence: retorna ultimo bit da sequencia
    * first_bit_sequence: retorna o primeiro bit da sequencia
    * return_confirmation_frame

In [3]:
class ConfirmationFrame:
    __delimiter = None
    __sequence = None
    __destination_address = None
    __source_address = None

    def __init__(self, __delimiter, sequence, destination_address, source_address):
        self.__delimiter = "01111110"
        self.__sequence = sequence
        self.__destination_address = destination_address
        self.__source_address = source_address

    def last_bit_sequence(self):
        return self.__sequence[7]

    def first_bit_sequence(self):
        return self.__sequence[0]

    def return_confirmation_frame(self):
        frame = ''
        frame += self.__delimiter
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address


### Classe Conversion
    * decimal_to_binary : transforma decimal para binario
    * binary_to_decimal: transforma binario para decimal
    * binary_to_hexadecimal: transforma binario para hexadecimal
    * hexadecimal_to_string: transforma hexadecimal para string
    * binary_to_string: transforma binario para string
    * ip_to_binary: transforma ip em binario 

In [4]:
class Conversion:

    @staticmethod
    def decimal_to_binary(decimal):
        temp = ""
        while decimal > 0:
            s = str(int(decimal % 2))
            temp += s
            decimal = int(decimal / 2)
        if len(temp) < 8:
            s = ""
            i = 0
            size = len(temp)
            loop = 8 - size
            while i < loop:
                s += "0"
                i += 1
            temp = temp + s
        binary = ""
        i = 0
        while i < 8:
            binary += str(temp[7 - i])
            i += 1
        return binary

    @staticmethod
    def decimal_to_hexadecimal(decimal):
        pass

    @staticmethod
    def binary_to_decimal(binary):
        decimal = 0
        for bit in binary:
            if bit == "1":
                decimal = decimal * 2 + 1
            else:
                decimal = decimal * 2
        return decimal

    @staticmethod
    def binary_to_hexadecimal(binary):
        return hex(int(binary, 2))

    @staticmethod
    def hexadecimal_to_string(hexadecimal):
        return bytearray.fromhex(hexadecimal).decode('ascii')

    @staticmethod
    def binary_to_string(bin):
        hex = Conversion.binary_to_hexadecimal(bin)
        s = Conversion.hexadecimal_to_string(hex[2:])
        return s

    @staticmethod
    def hexadecimal_to_binary(hexadecimal):
        a = bin(int(hexadecimal, 16))
        return a.replace('b', '')

    @staticmethod
    def string_to_hexadecimal(string):
        return string.encode('ascii').hex()

    @staticmethod
    def string_to_binary(string):
        string = Conversion.string_to_hexadecimal(string)
        return Conversion.hexadecimal_to_binary(string)

    @staticmethod
    def ip_to_binary(ip):
        ip = ip.split('.')
        binary = ""
        for num in ip:
            binary += Conversion.decimal_to_binary(int(num))
        return binary


### Classe CRC
    * crc: calcula o CRC da sequencia de bits

In [5]:
class CRC:

    @staticmethod
    def crc(msg):

        div = '11000000000000101'
        code = (len(div) - 1) * '0'
        msg = msg + code

        msg = list(msg)
        div = list(div)

        for i in range(len(msg) - len(code)):
            if msg[i] == '1':
                for j in range(len(div)):
                    msg[i + j] = str((int(msg[i + j]) + int(div[j])) % 2)

        return ''.join(msg[-len(code):])


### Classe Enlace
    * data_request: acesso do servidor a camada de enlace
    * data_indication: acesso do usuario a camada de enlace
    

In [6]:
from DataFrame import DataFrame
from Conversion import Conversion
from MySocket import MySocket
from CRC import CRC


class Enlace:

    def __init__(self, address):
        __destination_address = address[0]
        __source_address = address[1]

    @staticmethod
    def data_request():
        socket = MySocket()
        respostas = socket.receive()
        mensagem = ''
        for r in respostas:
            r = r[1:]
            r2 = DataFrame.string_to_DataFrame(r)
            r2_bin = r2.get_payload()
            mensagem += Conversion.binary_to_string(r2_bin)

        return (mensagem)

    @staticmethod
    def data_indication(destination_address, source_address, l_sdu):

        # Delimitador já é inserido no quadro

        # comprimento do payload
        length = Conversion.decimal_to_binary(len(l_sdu))

        # campo de sequencia TODO
        #          SEQ----ACK
        sequence = '00000000'

        # destino do quadro
        destination_address_frame = Conversion.ip_to_binary(destination_address)

        # origem do quadro
        source_address_frame = Conversion.ip_to_binary(source_address)

        # dados convertidos
        payload = Conversion.string_to_binary(l_sdu)

        frame = DataFrame(
            length,
            sequence,
            destination_address_frame,
            source_address_frame,
            payload)

        MyCRC = CRC.crc(frame.return_data_frame())
        frame.set__crc(MyCRC)

        socket = MySocket(destination_address)
        if l_sdu == 'FIM':
            socket.send('#')
        else:
            print(frame.return_data_frame())
            socket.send(frame.return_data_frame())


### Classe User
    * send_message: envia mensagem
    * receive_message: recebe mensagem

In [7]:
from Enlace import Enlace
import math


class User:

    def __init__(self, host):
        self.__host = host

    def send_message(self, message, destination_address):
        npackets = int(math.ceil(len(message) / 255))
        listpackets = []
        for i in range(npackets):
            listpackets.append(message[i * 255:(i + 1) * 255])

        for segment in listpackets:
            Enlace.data_indication(destination_address, self.__host, segment)
        Enlace.data_indication(destination_address, self.__host, 'FIM')

    def receive_message(self):
        message = Enlace.data_request()

        print(message)


### Main

In [ ]:
from User import User

import socket

HOST_SOURCE = "127.0.0.1"
HOST_DESTINATION = HOST_SOURCE


def main():
    usuario = User(HOST_SOURCE)

    usuario.receive_message()


if __name__ == '__main__':
    main()
